# Generate async request JSON file for ChatComplete API: User Profile 

In [20]:
import pandas as pd
from tqdm.notebook import tqdm
import json
import re
from collections import OrderedDict
import numpy as np
import pickle

In [2]:
import openai
import os
with open("../specs/.openai_api_key", "r") as f:
    ss = f.readline().strip()

os.environ["OPENAI_API_KEY"] = ss

In [3]:
from openai import OpenAI
client = OpenAI()

In [4]:
from openai import OpenAI
client = OpenAI()

model_list = client.models.list()

In [5]:
for x in model_list: print(x)

Model(id='dall-e-3', created=1698785189, object='model', owned_by='system')
Model(id='gpt-4-1106-preview', created=1698957206, object='model', owned_by='system')
Model(id='dall-e-2', created=1698798177, object='model', owned_by='system')
Model(id='tts-1', created=1681940951, object='model', owned_by='openai-internal')
Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='system')
Model(id='tts-1-1106', created=1699053241, object='model', owned_by='system')
Model(id='tts-1-hd', created=1699046015, object='model', owned_by='system')
Model(id='text-embedding-3-large', created=1705953180, object='model', owned_by='system')
Model(id='babbage-002', created=1692634615, object='model', owned_by='system')
Model(id='gpt-4-turbo-preview', created=1706037777, object='model', owned_by='system')
Model(id='gpt-4-0125-preview', created=1706037612, object='model', owned_by='system')
Model(id='gpt-4o-mini', created=1721172741, object='model', owned_by='system')
Model(id='gpt-4o-mini-20

In [6]:
# engine_id='gpt-3.5-turbo-16k'
engine_id='gpt-4o-mini'

In [ ]:
def make_json_body(request_id, system_prompt, user_prompt):
    dic = { 
        'model':engine_id,
        'messages':[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        'temperature':0.0,
        'metadata':{'request_id':request_id},
    }
    return json.dumps(dic, ensure_ascii=False)

## The prompt temmplate for generating User Profile 

In [ ]:
system_prompt = """
You will serve as an assistant to help me determine which types of products a specific user is likely to buy.
I will provide you with information about products that the user has purchased, as well as his or her reviews of those products.
Here are the instructions:
1. Each purchased product will be described in JSON format, with the following attributes:
{
    "title": "the title of the product in Korean", (if there is no title, I will set this value to "None")
    "description": "a description of what types of users will like this product in Korean",(if there is no description, I will set this value to "None"),
    "review": "the user's review on the product" (if there is no review, I will set this value to "None")
}

2. The information I will give you:
PURCHASED PRODUCTS: a list of JSON strings describing the items that the user has purchased.

Requirements:
1. Please provide your decision in JSON format, following this structure:
{
    "summarization": "A summarization of what types of products this user is likely to enjoy" (if you are unable to summarize it, please set this value to "None")
    "reasoning": "briefly explain your reasoning for the summarization"
}
2. Please ensure that the "summarization" is no longer than 200 words.
3. The "reasoning" has no word limits.
4. Do not provided any other text outside the JSON string.
"""

In [ ]:
compact_history_df = pd.read_pickle('/Jupyter/dev_src/RLMRec/data/katchers/intermediate/compact_history_df.pkl')

In [ ]:
api_request_file_path = '/Jupyter/dev_src/RLMRec/data/katchers/intermediate/openai-request-users.json'

In [ ]:
with open(api_request_file_path, 'w') as f:
    cnt = 0
    
    for _, row in compact_history_df.iterrows():
        jsonl = make_json_body(
            request_id=f"{row.name}",#cnt+i,
            system_prompt=system_prompt,
            user_prompt=row['user_profile_input']
            )      
        f.write(jsonl + '\n')

## Analysis generated responses

In [7]:
response_file_path = '/Jupyter/dev_src/RLMRec/data/katchers/intermediate/openai-response-users.json'

In [8]:
in_fname =  response_file_path
datas = []
request_ids = []
# results = {}
with open(in_fname, 'r') as f:
    for i, line in enumerate(f):
        data = json.loads(line)
        if isinstance(data[1] , dict):
            # results[i] = data
            datas.append(data[1]['choices'][0]['message']['content'])
            request_ids.append(data[2]['request_id'])

In [9]:
df = pd.DataFrame({'request_id': request_ids, 'response': datas})
df['response'] = df['response'].str.strip()
df['uid'] = df.request_id.astype(int)
profile_df = df.sort_values(by='uid')

# Generate aynsc request JSON file for Embedding API

In [10]:
profile_df

,request_id,response,uid
13450,0,"{\n ""summarization"": ""이 사용자는 주방용품, 개인 위생 제품...",0
10084,1,"{\n ""summarization"": ""이 사용자는 신선한 농산물과 가공육 제...",1
55,2,"{\n ""summarization"": ""이 사용자는 신선한 채소와 반찬을 선호...",2
13451,3,"{\n ""summarization"": ""이 사용자는 간편하고 건강한 식품, 특...",3
16,4,"{\n ""summarization"": ""이 사용자는 청소 및 세탁 관련 제품과...",4
...,...,...,...
16808,16808,"{\n ""summarization"": ""이 사용자는 주름 개선 및 피부 관리 ...",16808
16807,16809,"{\n ""summarization"": ""이 사용자는 주름 개선 및 피부 관리 ...",16809
13445,16810,"{\n ""summarization"": ""이 사용자는 주름 개선과 피부 관리에 ...",16810
16810,16811,"{\n ""summarization"": ""이 사용자는 주름 개선 및 피부 관리 ...",16811


In [11]:
engine_id='text-embedding-3-small'

In [12]:
def make_json_body(engine_id, idx, body):
    dic = { 
        'model':engine_id,
        'input':body,
        'metadata': {"request_id": idx}
    }
    return json.dumps(dic, ensure_ascii=False)

In [13]:
api_request_file_path = '/Jupyter/dev_src/RLMRec/data/katchers/intermediate/openai-request-users-embed.json'

In [14]:
with open(api_request_file_path, 'w') as f:
    cnt = 0
    
    for _, row in profile_df.iterrows():
        jsonl = make_json_body(
            engine_id=engine_id,
            idx = row['uid'],
            body=row['response']
            )      
        f.write(jsonl + '\n')

## Analysis generated responses

In [15]:
generated_data_path = '/Jupyter/dev_src/RLMRec/data/katchers/intermediate/openai-response-users-embed.json'

In [16]:
in_fname = generated_data_path
datas = []
dic = OrderedDict()
with open(in_fname, 'r') as f:
    for i, line in enumerate(f):
        data = json.loads(line)
        
        IDX = data[2]['request_id']
        embedding = data[1]['data'][0]['embedding']
        
        dic[IDX] = embedding
result_df = pd.DataFrame.from_records([{'IDX':k, 'embedding':v} for k, v in dic.items()]).sort_values('IDX')
result_df.set_index('IDX', inplace=True)

# Write User Embedding file

In [19]:
embedding_matrix = np.vstack(result_df['embedding'].values)

In [21]:
with open('/Jupyter/dev_src/RLMRec/data/katchers/usr_emb_np.pkl', 'wb') as file:
    pickle.dump(embedding_matrix, file)

In [22]:
embedding_matrix.shape

(16813, 1536)